In [11]:
from typing import NamedTuple
from enum import Enum
from dataclasses import dataclass

import matplotlib.pyplot as plt
import utils

## Day 2: Rock Paper Scissors

[#](https://adventofcode.com/2022/day/2) 

* Rock defeats Scissors, Scissors defeats Paper, and Paper defeats Rock. Same is a draw.

Strategy guide:
* 1st col: A for Rock, B for Paper, and C for Scissors
* 2nd col: X for Rock, Y for Paper, and Z for Scissors

Score for 1 round by adding the score for shape and outcome:
* Shape you selected: 1 for Rock, 2 for Paper, 3 for Scissors
* Outcome: 0 if you lost, 3 if the round was a draw, and 6 if you won)

In [12]:
test: str = """A Y
B X
C Z"""

inp_1 = utils.get_input(2)

First up, to parse the input I'm making a list of lists, using the same list comprehension as day 1:

In [13]:
[[n for n in y.split(" ")] for y in test.strip().splitlines()]

[['A', 'Y'], ['B', 'X'], ['C', 'Z']]

That works well, so putting it in a function:

In [14]:
def parse(inp=test, verbose=False):
    """parses input and returns a list of actions"""
    return [[n for n in y.split(" ")] for y in inp.strip().splitlines()]

data = parse()
data

[['A', 'Y'], ['B', 'X'], ['C', 'Z']]

Now we need a way to translate the code letters into rocks, papers and scissors, and get a score for the objects. 
So this is just a simple lookup table, so I've gone with dicts. If there was some logic needed here dataclasses would be more appropriate.

To relearn some python tricks, here goes a dict comprehension. With three objects its less readable than just writing out the dict properly, but this was useful as I could make one list, and resuse it in three dicts, ensuring that the spellings were consistent, and I saved a few chars of typing. 

In [21]:
shapes = ["Rock", "Paper", "Scissors"]

shape_score = dict(zip(shapes, [1,2,3]))
print(f"shapes with scores: {shape_score}")

cypher_1 = dict(zip(["A", "B", "C"], shapes))
print(f"cypher for col1: {cypher_1}")

cypher_2 = dict(zip(["X", "Y", "Z"], shapes))
print(f"cypher for col1: {cypher_2}")

shapes with scores: {'Rock': 1, 'Paper': 2, 'Scissors': 3}
cypher for col1: {'A': 'Rock', 'B': 'Paper', 'C': 'Scissors'}
cypher for col1: {'X': 'Rock', 'Y': 'Paper', 'Z': 'Scissors'}


In [38]:
def game(moves):
    """plays paper rock scissors and returns a score for player 2"""
    # the move each player makes
    p1 = cypher_1[moves[0]]
    p2 = cypher_2[moves[1]]

    score: int = shape_score[p2]
    # start with assuming a loss
    win: bool = False

    # check for draw
    if p1 == p2:
        return score + 3

    # working through the game from the view of p2 (us)
    match p2:
        case "Rock":
            if p1 == "Scissors":
                win = True
        case "Paper":
            if p1 == "Rock":
                win = True
        case "Scissors":
            if p1 == "Paper":
                win = True

    return score + (6 * win)


for moves in data:
    print(moves, game(moves))

['A', 'Y'] 8
['B', 'X'] 1
['C', 'Z'] 6


Putting the above together:

In [39]:
def play_games(inp=test):
    total_score = 0
    moves = parse(inp)
    scores = [game(move) for move in moves]
    
    return sum(scores)

assert play_games(test) == 15
play_games(inp_1)

9651

## Part 2

Now instead of the cypher just being a cypher, the second col is telling us the outcome, and we figure out what to play.

> X means you need to lose, Y means you need to end the round in a draw, and Z means you need to win.

So updating the cypher for col 2:

In [34]:
cypher_3 = {
    "X": "Loose",
    "Y": "Draw",
    "Z": "Win",
}

An easier way to solve part 2 is to just play the game for each shape, and select the appropirate score - Loose is index 0, Draw is index 1 and Win is index 2. 

The below just essentially hard codes the answers:

In [58]:
def game_2(moves):
    """plays paper rock scissors and returns a score for player 2"""
    p1 = cypher_1[moves[0]]
    p2 = cypher_3[moves[1]]

    score: int = 0
    # start with assuming a loss
    win: bool = False

    match p2:
        case "Draw":
            score += shape_score[p1] + 3

        case "Loose":
            match p1:
                case "Rock":
                    score += shape_score["Scissors"]
                case "Paper":
                    score += shape_score["Rock"]
                case "Scissors":
                    score += shape_score["Paper"]

        case "Win":
            win = True
            match p1:
                case "Rock":
                    score += shape_score["Paper"]
                case "Paper":
                    score += shape_score["Scissors"]
                case "Scissors":
                    score += shape_score["Rock"]
            
    
    #print(p1, p2, win + 6*win, score)
    
    return score + (6 * win)


for moves in data:
    print(moves, game_2(moves))

['A', 'Y'] 4
['B', 'X'] 1
['C', 'Z'] 7


Putting it together:

In [59]:
def play_game_2(inp=test):
    total_score = 0
    moves = parse(inp)
    scores = [game_2(move) for move in moves]
    
    return sum(scores)

assert play_game_2(test) == 12
play_game_2(inp_1)

10560